In [19]:
import pandas as pd
import torch

Data = pd.read_csv("food_wastage_classification_dataset_total.csv")
Data = Data.drop(["Category"],axis=1)
Data

,Sales,Stock Remaining (%),Avg_consumption Rate (%),Price,AVC,Label
0,93,0.07,0.75,13.55,5.15,0
1,100,0.00,1.00,8.85,3.92,0
2,81,0.19,0.77,17.73,7.54,0
3,100,0.00,0.99,45.11,24.98,0
4,99,0.01,0.88,19.97,6.71,0
...,...,...,...,...,...,...
99995,4,0.96,0.00,14.16,7.33,3
99996,4,0.96,0.01,15.76,8.84,3
99997,3,0.97,0.00,10.29,5.30,3
99998,3,0.97,0.20,17.91,7.25,3


In [20]:
X = Data.drop(["Label"], axis=1)
X = torch.tensor(X.values, dtype=torch.float32)

Y = Data["Label"]
Y = torch.tensor(Y, dtype=torch.long)

In [21]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 64

class FoodDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):        
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [22]:
Dataset = FoodDataset(X, Y)
TrainingData, TestingData, ValidationData = random_split(Dataset, [0.7, 0.2, 0.1])

TrainLoader      = DataLoader(TrainingData, batch_size=BATCH_SIZE, shuffle=True)
TestLoader       = DataLoader(TestingData, batch_size=BATCH_SIZE, shuffle=False)
ValidationLoader = DataLoader(ValidationData, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

class Stephen(nn.Module):
    def __init__(self):
        super().__init__()

        self.FinanceConsultant = nn.Sequential(
            nn.Linear(5, 16),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4)
        )

        self.StephenDict  = {0 : 1,    1 : 0.5,  2 : -0.5,  3 : -1}
        self.PriceChanges = {0 : 0.10, 1 : 0.05, 2 : -0.05, 3 : -0.10}
        
    def forward(self, x):
        x = self.FinanceConsultant(x)
        return x

    def adjust_price(self, Price, x):
        Advice = torch.argmax(self.FinanceConsultant(x))
        StephensOpinion = self.StephenDict.get(Advice.item(), 0)
        
        AdjustedPrice = Price + (Price * self.PriceChanges.get(StephensOpinion, 0))

        return max(AdjustedPrice, 1)
    
    def predict(self, x):
        return torch.argmax(self.FinanceConsultant(x))
    

In [24]:
from torchsummary import summary
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

Consultant      = Stephen()
Optimizer       = Adam(Consultant.parameters(), lr=0.001, weight_decay=0.01)
CostFunction    = nn.CrossEntropyLoss()
Scheduler       = ExponentialLR(Optimizer, gamma=0.5) # To prevent overfitting the model we will use this to exponentially decrease the learning rate every epoch

In [25]:
from tqdm import tqdm # Another library which just shows us % done of loops

# Training Loop
NUMEPOCHS = 7

for i in range(NUMEPOCHS):
    # Training Loop
    Consultant.train()
    TrainingLoss     = 0
    TrainingAccuracy = 0

    for batch in tqdm(TrainLoader, desc="Training"):
        x, y = batch
        
        # Do the forwards pass on our model to find out the error
        y_hat = Consultant(x)

        # Calculate the cost
        Cost = CostFunction(y_hat, y)

        # Backwards pass
        Optimizer.zero_grad()
        Cost.backward()
        Optimizer.step()

        # Record Loss and Accuracy
        TrainingLoss += Cost.item()
        TrainingAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    TrainingAccuracy /= len(TrainLoader.dataset)
    TrainingLoss     /= len(TrainLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Training Loss: {(TrainingLoss / len(TrainLoader)):.4f}, Training Accuracy {TrainingAccuracy * 100}%")

    # For Validation loop it's pretty much the same
    Consultant.eval()
    ValidationLoss     = 0
    ValidationAccuracy = 0

    with torch.no_grad():
        for batch in tqdm(ValidationLoader, desc="Validating"):
            x, y = batch

            # Do the forwards pass on our model to find out the error
            y_hat = Consultant(x)

            # Calculate the cost
            ValidationCost = CostFunction(y_hat, y)

            # Record loss and Accuracy
            ValidationLoss += ValidationCost.item()
            ValidationAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    ValidationAccuracy /= len(ValidationLoader.dataset)
    ValidationLoss     /= len(ValidationLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Validation Loss: {(ValidationLoss):.4f}, Validation Accuracy { ValidationAccuracy * 100}%")

    # Adjust Learning Rate
    Scheduler.step()


Training: 100%|██████████| 1094/1094 [00:01<00:00, 726.54it/s]


Epoch 1/7, Avg. Training Loss: 0.0005, Training Accuracy 74.02142857142857%


Validating: 100%|██████████| 157/157 [00:00<00:00, 1634.19it/s]


Epoch 1/7, Avg. Validation Loss: 0.3127, Validation Accuracy 88.38000000000001%


Training: 100%|██████████| 1094/1094 [00:01<00:00, 741.38it/s]


...

Epoch 6/7, Avg. Validation Loss: 0.1797, Validation Accuracy 94.88%


Training: 100%|██████████| 1094/1094 [00:01<00:00, 793.44it/s]


Epoch 7/7, Avg. Training Loss: 0.0002, Training Accuracy 94.72142857142856%


Validating: 100%|██████████| 157/157 [00:00<00:00, 1562.19it/s]

Epoch 7/7, Avg. Validation Loss: 0.1787, Validation Accuracy 94.8%


In [26]:
Consultant.eval()
TestingLoss     = 0
TestingAccuracy = 0

with torch.no_grad():
    for batch in tqdm(TestLoader, desc="Testing"):
        x, y = batch

        # Do the forwards pass on our model to find out the error
        y_hat = Consultant(x)

        # Calculate the cost
        TestingCost = CostFunction(y_hat, y)

        # Record loss and Accuracy
        TestingLoss += TestingCost.item()
        TestingAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()

TestingAccuracy /= len(TestLoader.dataset)
TestingLoss     /= len(TestLoader)

print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Testing Loss: {(TestingLoss):.4f}, Testing Accuracy { TestingAccuracy * 100}%")

Testing: 100%|██████████| 313/313 [00:00<00:00, 1388.63it/s]

Epoch 7/7, Avg. Testing Loss: 0.1774, Testing Accuracy 94.76%


In [ ]:
torch.save(Consultant.state_dict(), "TrainedModels/Stephen.pth", )